In [1]:
import os
from openai import AzureOpenAI
from string import Template
import json
from neo4j import GraphDatabase
import glob
from timeit import default_timer as timer
from dotenv import load_dotenv
from time import sleep

load_dotenv()

True

In [2]:
# Neo4j configuration & constraints
neo4j_url = os.getenv("NEO4J_CONNECTION_URL")
neo4j_user = os.getenv("NEO4J_USER")
neo4j_password = os.getenv("NEO4J_PASSWORD")
gds = GraphDatabase.driver(neo4j_url, auth=(neo4j_user, neo4j_password))

In [ ]:
# Function to take a json-object of entitites and relationships and generate cypher query for creating those entities
def generate_cypher(extractions):
    e_statements = []
    r_statements = []

    e_label_map = {}

    r_failed = []

    for i, extraction in enumerate(extractions):
        print(f"Generating cypher for chunk {i+1} of {len(extractions)}")
        chunk_id = extraction["chunkId"]
        chunk_cypher = f"MERGE (n:Chunk {{id: '{chunk_id}'}})"
        e_statements.append(chunk_cypher)
        for entity in extraction["entities"]:
            label = entity["label"].replace(" ", "_")
            name = entity["name"].replace(" ", "_")

            cypher = f"MERGE (n:{label} {{name: '{name}'}})"
            connect_chunk_cypher = f"MERGE (a:Chunk{{id: '{chunk_id}'}}) MERGE (b:{label} {{name: '{name}'}}) MERGE (b)-[:EXTRACTED_FROM]->(a)"
            e_statements.append(cypher)
            e_statements.append(connect_chunk_cypher)
            e_label_map[name] = label

        for rs in extraction["relationships"]:
            try:
                head, rs_type, tail = rs.split("|")
            except:
                r_failed.append(rs)
                continue

            head = head.replace(" ", "_")
            tail = tail.replace(" ", "_")

            try:
                head_label = e_label_map[head]
                target_label = e_label_map[tail]
                cypher = f"MERGE (a:{head_label} {{name: '{head}'}}) MERGE (b:{target_label} {{name: '{tail}'}}) MERGE (a)-[:{rs_type}]->(b)"
                r_statements.append(cypher)
            except:
                r_failed.append(rs + f" ({i})")

    with open("r_failed.txt", "w") as f:
        f.write(f"{r_failed}")
    return e_statements + r_statements


# Function to apply cypher and create knowlegde graph with Neo4j
def apply_cypher(cypher_statements):
    for i, stmt in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}")
        try:
            gds.execute_query(stmt)
        except Exception as e:
            with open("failed_statements.txt", "w") as f:
                f.write(f"{stmt} - Exception: {e}\n")

In [39]:
with open("data/es_n_rs_all.json") as f:
    extractions = json.load(f)

cypher_statements = generate_cypher(extractions)

apply_cypher(cypher_statements)

# with open("cypher_:20.txt", "w") as f:
#     f.write("\n".join(cypher_statements))

Generating cypher for chunk 1 of 528
Generating cypher for chunk 2 of 528
Generating cypher for chunk 3 of 528
Generating cypher for chunk 4 of 528
Generating cypher for chunk 5 of 528
Generating cypher for chunk 6 of 528
Generating cypher for chunk 7 of 528
Generating cypher for chunk 8 of 528
Generating cypher for chunk 9 of 528
Generating cypher for chunk 10 of 528
Generating cypher for chunk 11 of 528
Generating cypher for chunk 12 of 528
Generating cypher for chunk 13 of 528
Generating cypher for chunk 14 of 528
Generating cypher for chunk 15 of 528
Generating cypher for chunk 16 of 528
Generating cypher for chunk 17 of 528
Generating cypher for chunk 18 of 528
Generating cypher for chunk 19 of 528
Generating cypher for chunk 20 of 528
Generating cypher for chunk 21 of 528
Generating cypher for chunk 22 of 528
Generating cypher for chunk 23 of 528
Generating cypher for chunk 24 of 528
Generating cypher for chunk 25 of 528
Generating cypher for chunk 26 of 528
Generating cypher for

TypeError: write() argument must be str, not list